In [16]:
# pip install v4-proto
# pip install v4-client-py
# pip uninstall v4-client-py (yes really)
# git clone https://github.com/kaloureyes3/v4-clients
# pip install python-dotenv
# pip install web3

# import requests
from web3 import Web3
import os
import time
from dotenv import load_dotenv
import sys
sys.path.insert(0, os.path.dirname(os.path.abspath(".env"))+'/v4-clients/v4-client-py')

from v4_client_py import clients
from v4_client_py.clients import constants, IndexerClient, CompositeClient, Subaccount
from v4_client_py.clients.constants import Network, CHAIN_ID, IndexerConfig, ValidatorConfig
from v4_client_py.chain.aerial.wallet import LocalWallet


In [17]:
# set up client and variables
load_dotenv()
# variables in .env need to be = DYDXWALLETADDRESS




public_client = IndexerClient(
    config = IndexerConfig(
        rest_endpoint='https://indexer.dydx.trade',
        websocket_endpoint='wss://indexer.dydx.trade',
    ),
)
network = Network.config_network()
private_client = CompositeClient(
    network,
)

dydxWalletAddress = os.getenv("DYDXWALLETADDRESS")
dydxSecretPhrase = os.getenv("DYDXSECRETPHRASE")

In [18]:
wallet = LocalWallet.from_mnemonic(dydxSecretPhrase)
subaccount = Subaccount(wallet, 0)



In [14]:




# Request only API
# get all markets
# r = requests.get(f'{baseUrl}/perpetualMarkets', headers= headers).json()
# for i in r['markets']:
#     print(i)
# account info including freeCollateral
# r = requests.get(f'{baseUrl}/addresses/{dydxWalletAddress}', headers= headers).json()
# print (r['subaccounts'][0]['freeCollateral'])

#  TO BE CHECKED
# list positions
# r = requests.get(f'{baseUrl}/perpetualPositions', params={'address': dydxWalletAddress, 'subaccountNumber': 0, 'status': ["OPEN"], 'limit': 100}, headers= headers).json()
# print (r)
# list orders
# r = requests.get(f'{baseUrl}/orders', params = {'address': dydxWalletAddress, 'subaccountNumber': 0, 'limit': 100}, headers= headers).json()
# get one order
# orderId = "12345"
# r = requests.get(f'{baseUrl}/orders/{orderId}', headers= headers).json()





In [4]:
# main functions
def getWtAvePriceDyDx3(size, orderbook):
    sizeLeft = float(size)
    wtAvePx = 0
    for entry in orderbook:
        if sizeLeft <= 0:
            break
        sizeUsed = min(sizeLeft, float(entry["size"]))
        wtAvePx += float(entry["price"]) * sizeUsed
        sizeLeft -= sizeUsed
    return (wtAvePx / (size - sizeLeft)) # just in case the order is bigger than the market depth

def getBidOfferForSizeDyDx3(size, orderbook):
    if (size <= 0):
        size = 1
    return (getWtAvePriceDyDx3(size, orderbook["bids"]), getWtAvePriceDyDx3(size, orderbook["asks"]))


In [20]:
# res = public_client.markets.get_perpetual_markets().data
# print(res["markets"]["ETH-USD"]) # get all markets

# res = public_client.markets.get_perpetual_market_orderbook("ETH-USD").data # get orderbook (e.g. ETH-USD)
# getBidOfferForSizeDyDx3(10, res) # get the bid and offer in 10

# res = public_client.account.get_subaccount(dydxWalletAddress,0)
# print(res.data["subaccount"]["freeCollateral"]) # get available collateral
# also open positions

# get open orders
# res = public_client.account.get_subaccount_orders(dydxWalletAddress, 0).data
# print(res)

# cancel an order
tx = private_client.cancel_order(subaccount, client_id=593631309, market="ETH-USD", order_flags=64, good_til_block=0, good_til_time_in_seconds=0)



_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNIMPLEMENTED
	details = "Received http2 header with status: 404"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2024-03-16T23:00:22.6050474+00:00", grpc_status:12, grpc_message:"Received http2 header with status: 404"}"
>

In [13]:
print(res)

[{'id': '43668817-4023-585f-8f46-43203ce19a82', 'subaccountId': 'ab8b1635-d329-54d2-8baf-ff314b43d01e', 'clientId': '593631309', 'clobPairId': '1', 'side': 'BUY', 'size': '0.1', 'totalFilled': '0', 'price': '3001', 'type': 'LIMIT', 'status': 'OPEN', 'timeInForce': 'GTT', 'reduceOnly': False, 'orderFlags': '64', 'goodTilBlockTime': '2024-04-13T16:04:21.000Z', 'createdAtHeight': '10967315', 'clientMetadata': '0', 'updatedAt': '2024-03-16T16:04:20.150Z', 'updatedAtHeight': '10967315', 'postOnly': False, 'ticker': 'ETH-USD'}, {'id': '9348fa2c-34ed-560e-94b8-e00190613ffb', 'subaccountId': 'ab8b1635-d329-54d2-8baf-ff314b43d01e', 'clientId': '1254633427', 'clobPairId': '1', 'side': 'SELL', 'size': '0.1', 'totalFilled': '0.1', 'price': '3490', 'type': 'LIMIT', 'status': 'FILLED', 'timeInForce': 'FOK', 'reduceOnly': False, 'orderFlags': '0', 'goodTilBlock': '10967339', 'createdAtHeight': '10967338', 'clientMetadata': '1', 'updatedAt': '2024-03-16T16:04:45.059Z', 'updatedAtHeight': '10967338', '